In [1]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [2]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [3]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.6848, Test: 0.7024
Epoch: 002, Loss: 0.6822, Val: 0.6717, Test: 0.6972
Epoch: 003, Loss: 0.7112, Val: 0.6837, Test: 0.7036
Epoch: 004, Loss: 0.6782, Val: 0.7147, Test: 0.7223
Epoch: 005, Loss: 0.6855, Val: 0.7470, Test: 0.7503
Epoch: 006, Loss: 0.6891, Val: 0.7717, Test: 0.7649
Epoch: 007, Loss: 0.6901, Val: 0.7537, Test: 0.7450
Epoch: 008, Loss: 0.6897, Val: 0.7288, Test: 0.7210
Epoch: 009, Loss: 0.6882, Val: 0.7128, Test: 0.7068
Epoch: 010, Loss: 0.6849, Val: 0.6986, Test: 0.6978
Epoch: 011, Loss: 0.6802, Val: 0.6890, Test: 0.6927
Epoch: 012, Loss: 0.6761, Val: 0.6815, Test: 0.6883
Epoch: 013, Loss: 0.6781, Val: 0.6771, Test: 0.6841
Epoch: 014, Loss: 0.6753, Val: 0.6737, Test: 0.6811
Epoch: 015, Loss: 0.6682, Val: 0.6705, Test: 0.6800
Epoch: 016, Loss: 0.6650, Val: 0.6713, Test: 0.6841
Epoch: 017, Loss: 0.6616, Val: 0.6702, Test: 0.6897
Epoch: 018, Loss: 0.6574, Val: 0.6669, Test: 0.6910
Epoch: 019, Loss: 0.6486, Val: 0.6630, Test: 0.6896
Epoch: 020, 

In [5]:
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    2,    7,  ..., 2705, 2706, 2707]], device='cuda:0')